In [49]:
import numpy as np
import pandas as pd
import re, gc
import time, requests
from datetime import datetime, timedelta
import calendar

import sqlalchemy as sql
from sqlalchemy.pool import NullPool
from urllib.parse import quote
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# !pip install mysqlclient==2.1.1

In [50]:
import datetime
import calendar

def monthdelta(date, delta):
    m, y = (date.month + delta) % 12, date.year + ((date.month) + delta - 1) // 12
    if not m:
        m = 12
    d = min(date.day, [31, 29 if y % 4 == 0 and (not y % 100 == 0 or y % 400 == 0) else 28,
                       31, 30, 31, 30, 31, 31, 30, 31, 30, 31][m - 1])
    return date.replace(day=d, month=m, year=y)

today = datetime.datetime.today()

# Get the start period for the transaction (one month ago from today)
start_period = monthdelta(today, -1).replace(day=1).date()

# Get the end period for the transaction (last day of one month ago from today)
end_period = monthdelta(today, -1)
end_period = end_period.replace(day=calendar.monthrange(end_period.year, end_period.month)[1]).date()

print("start_period =", start_period.strftime("%Y-%m-%d"))
print("end_period =", end_period.strftime("%Y-%m-%d"))


start_period = 2023-06-01
end_period = 2023-06-30


In [51]:
username = ""
password = ""
host = ""

connect_string_segtor = 'mysql://'+username+':'+quote(password)+'@'+host+'/?charset=utf8mb4'
sql_engine_segtor = sql.create_engine(connect_string_segtor, poolclass=NullPool)
sql_engine_segtor.connect()

In [52]:

# dinamic1 use this because can provide all data exist
# SQL Query with start_period and end_period
q = f'''
SELECT segment_name, owner_email, approved
FROM db_segmentgenerator.segtor_jobs_v2
WHERE DATE(submit_date) >= '{start_period}'
AND DATE(submit_date) <= '{end_period}'
'''


df = pd.read_sql(q, sql_engine_segtor)

In [53]:
df1 = df.groupby('owner_email').size().reset_index().rename(columns={0: 'Tasks Created'})

In [54]:
df2 = df[df['approved'] == 1].drop_duplicates(subset=['segment_name']).groupby('owner_email').size().reset_index().rename(columns={0: 'Segments Uploaded'})

In [55]:
final_df = df1.merge(df2, how='left', on='owner_email').fillna(0)

## Rules

- Tasks Created means : the user created a segment (not approved yet), so it only receive 1 email

- Segments Uploaded	means: the user approved this segment, he/she will receive another <b>callback</b> email (+ 1), also send <b>campaign tagger</b> email to operation (+ 1)

------------------------------

Email Usage: Task Created + (Segments Uploaded * 2)

In [56]:
final_df['Segments Uploaded'] = final_df['Segments Uploaded'].astype(int)
final_df['Email Usage'] = final_df['Tasks Created'] + (final_df['Segments Uploaded'] * 2)
final_df = final_df.rename(columns={'owner_email': 'User Email'})

In [57]:
final_df

,User Email,Tasks Created,Segments Uploaded,Email Usage
0,agathabangun@gmail.com,80,16,112
1,anninditaap@gmail.com,10,9,28
2,design.samsoniteid@gmail.com,4,2,8
3,edwinaviandari98@gmail.com,2,1,4
4,fandy.ariyadi@gmail.com,11,0,11
5,indravivojadino1@gmail.com,3,2,7
6,mayaagi04@gmail.com,76,28,132
7,memeyofficemap@gmail.com,44,22,88
8,nadhifahwardahh@gmail.com,3,3,9
9,tiaramyas18@gmail.com,60,24,108


In [66]:

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side

# Save the DataFrame to an Excel file
month_name = start_period.strftime("%B")  # Get the month name (e.g., June, July, etc.)
file_name = r"C:\Users\GDA-User\Documents\Reyki\kerja\2023 by project\SEGTOR EMAIL\segtor-usage.ipynb"

# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active

# Add the DataFrame to the worksheet
for r in dataframe_to_rows(final_df, index=False, header=True):
    ws.append(r)

# Apply styling to all cells
header_font = Font(bold=True)
header_fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
border = Border(left=Side(border_style='thin', color='000000'),
                right=Side(border_style='thin', color='000000'),
                top=Side(border_style='thin', color='000000'),
                bottom=Side(border_style='thin', color='000000'))

for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.font = header_font
        cell.fill = header_fill
        cell.border = border

# Save the Excel file
wb.save(file_name)


In [60]:
import datetime
import smtplib
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
# Email configuration
sender_email = "dp-alert@notify.gtech.digital"
sender_password = ""
receiver_email = "reyki.seprianza@gtech.digital"
subject = f"Segtor_usage_{month_name}"
body = f"Dear Mas Andi, \nBerikut saya kirimkan segtor usage periode {month_name} \n"

# Excel attachment configuration
attachment_path = file_name

# Compose the email
msg = MIMEMultipart()
msg["From"] = sender_email
msg["To"] = receiver_email
msg["Subject"] = subject
# msg["Cc"] = cc_receiver_email

msg.attach(MIMEText(body, "plain"))

# Attach the Excel file
with open(attachment_path, "rb") as f:
    attachment = MIMEApplication(f.read(), _subtype="xlsx")
    attachment.add_header("Content-Disposition", "attachment", filename=attachment_path)
    msg.attach(attachment)

# Send the email
with smtplib.SMTP('smtpdm-ap-southeast-1.aliyun.com', 80) as server:
    # server.starttls()
    server.login(sender_email, sender_password)
    server.send_message(msg)
    print("Email sent successfully!")

Email sent successfully!
